In [1]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB

In [2]:
df_train = pd.read_csv('./movie_review_train.csv')
df_train.head()

,class,text
0,Pos,a common complaint amongst film critics is ...
1,Pos,whew this film oozes energy the kind of b...
2,Pos,steven spielberg s amistad which is bas...
3,Pos,he has spent his entire life in an awful litt...
4,Pos,being that it is a foreign language film with...


In [3]:
df_test = pd.read_csv('./movie_review_test.csv')

In [4]:
df_test.head()

,class,text
0,Pos,films adapted from comic books have had plent...
1,Pos,every now and then a movie comes along from a...
2,Pos,you ve got mail works alot better than it des...
3,Pos,jaws is a rare film that grabs your atte...
4,Pos,moviemaking is a lot like being the general m...


In [5]:
df_train['class'].value_counts()

Pos    800
Neg    800
Name: class, dtype: int64

In [6]:
df_train['class'] = df_train['class'].map({'Pos':1,'Neg':0}) 

In [7]:
df_train.sample(5)

,class,text
547,1,edward zwick s the siege raises more ques...
624,1,in may of 1977 just 2 years after steven sp...
666,1,through a spyglass i could see everythin...
1188,0,perhaps if the impostors didn t have so much ...
826,0,not a great twelve months for either of the p...


In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
vect = CountVectorizer(stop_words='english')

In [10]:
vect.fit(df_train.text)

CountVectorizer(stop_words='english')

In [11]:
len(vect.vocabulary_.keys())

35858

In [12]:
# we don't want to consider those (rare) words which have appeared only in 3% of the documents, 
# or say those (extremely common ones) which have appeared in 80% of the documents
filtered_vect = CountVectorizer(stop_words='english', min_df=.03, max_df=.8)
filtered_vect.fit(df_train.text)
len(filtered_vect.vocabulary_.keys())

1643

In [13]:
x_test_transformed = filtered_vect.transform(df_test.text)

In [14]:
type(x_test_transformed)

scipy.sparse.csr.csr_matrix

In [15]:
print(x_test_transformed.shape[0])

400


In [16]:
x_test_transformed.count_nonzero()

51663

In [17]:
x_train_transformed = filtered_vect.transform(df_train.text)

In [18]:
x_train_transformed.count_nonzero()

217396

In [23]:
df_test['class'] = df_test['class'].map({'Pos':1,'Neg':0}) 

In [24]:
from sklearn.naive_bayes import BernoulliNB

# instantiate bernoulli NB object
bnb = BernoulliNB()

# fit 
bnb.fit(x_train_transformed,df_train['class'])

# predict class
y_pred_class = bnb.predict(x_test_transformed)

# predict probability
y_pred_proba =bnb.predict_proba(x_test_transformed)

# accuracy
from sklearn import metrics
metrics.accuracy_score(df_test['class'], y_pred_class)


0.79

In [26]:
from sklearn.metrics import confusion_matrix
metrics.confusion_matrix(df_test['class'], y_pred_class)

array([[177,  23],
       [ 61, 139]], dtype=int64)